In [15]:
from TPNN.tools.CNNarchitecture import *
from TPNN.tools.activation_functions import *
import numpy as np

## Test net creation

In [16]:
# create net:
cnnLayer = CNNlayer((1, 10, 10), sigmoid, 2, (3,3), 2)
poolingLayer = MaxPoolingLayer((2,2), cnnLayer.output_shape)
reformatLayer = ReformatLayer(poolingLayer.output_shape)
denseLayer = DenseLayer(sigmoid, 3, reformatLayer.output_shape[0])
softmaxLayer = SoftmaxLayer(3)

net = Net()
net.add_layer(cnnLayer)
net.add_layer(poolingLayer)
net.add_layer(reformatLayer)
net.add_layer(denseLayer)
net.add_layer(softmaxLayer)
net.print_config()

Layers count: 5
layers configuration: 

CNN layer:
 input shape - (1, 10, 10); output shape - (2, 4, 4)
 cores: 
[0]:
     0.30285310693082 -0.6885210964792909 -0.15311050382868197 
     -0.8529275639290272 -0.654674081420211 0.4937263057628398 
     -0.6116599173477506 -0.25236893017032913 -0.03844474027764577 
[1]:
     -0.5683507108337222 0.10898294837165623 -0.9164095594426178 
     -0.0668993250881913 -0.8476551166842712 -0.7438504806190906 
     -0.5064700524882704 0.9352268848634595 0.8691746330462846 
 biases: 
     0.8672126308557151 0.8397711189686163 

MaxPooling layer
 input shape - (2, 4, 4); output shape - (2, 2, 2)
 core shape: (2, 2)

Reformat layer
 input shape - (2, 2, 2); output shape - (4,)

Dense layer
 neuron count: 3
 input shape - (4,); output shape - (3,)
 weight matrix:
     -0.2412992790770243 0.3215280846001498 -0.9325637485305232 
     0.7126880469281742 -0.8898907898972737 -0.34596174524427425 
     0.9237193865510749 0.21781970760408753 -0.361201000937304

In [17]:
input_data = np.random.rand(1, 10, 10)

print("input:")
print(input_data)

input:
[[[0.64262366 0.41639218 0.46731835 0.19895396 0.3074723  0.09704952
   0.54288676 0.93746991 0.93750569 0.91842354]
  [0.33469268 0.48122553 0.67339196 0.77126938 0.44697223 0.92060624
   0.9899826  0.17678853 0.72764746 0.9944406 ]
  [0.76537074 0.95648751 0.16415146 0.10634139 0.18774562 0.74100421
   0.51173169 0.7194448  0.70391235 0.05454997]
  [0.68313564 0.21489443 0.88402921 0.24145225 0.19326584 0.98515919
   0.57537624 0.12952241 0.90243102 0.32843987]
  [0.97479707 0.53187571 0.80271081 0.88950095 0.62906424 0.0114703
   0.59670275 0.71816776 0.52244046 0.35553264]
  [0.43775193 0.48748563 0.29257115 0.65436124 0.74283096 0.43342584
   0.52193433 0.0728796  0.80069388 0.36217879]
  [0.06484265 0.96914434 0.78633442 0.42093512 0.03397112 0.66281917
   0.29406366 0.28326348 0.73306239 0.66338309]
  [0.85318164 0.91473723 0.57416814 0.25549904 0.16468257 0.11208016
   0.16305193 0.32304468 0.93332129 0.47074885]
  [0.76921296 0.04982235 0.85921628 0.58166839 0.58254122 

In [18]:
print(net.get_output(input_data))

[0.46650679 0.30676544 0.22672776]


## Test CNN layer output

In [19]:
def print_info(input, net):
    print("input:")
    print(input)
    print("net config")
    net.print_config()

cnnLayer = CNNlayer((2,4,4), ident, 2, (2,2), 2)
cores = np.array([np.eye(2), np.eye(2)])
cnnLayer.set_cores(cores)
cnnLayer.set_biases(np.array([1, 1]))

net = Net()
net.add_layer(cnnLayer)

input = np.array([np.eye(4),np.eye(4)])

print_info(input, net)

input:
[[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]]
net config
Layers count: 1
layers configuration: 

CNN layer:
 input shape - (2, 4, 4); output shape - (2, 2, 2)
 cores: 
[0]:
     1.0 0.0 
     0.0 1.0 
[1]:
     1.0 0.0 
     0.0 1.0 
 biases: 
     1 1 



In [20]:
print(net.get_output(input))

[[[5. 1.]
  [1. 5.]]

 [[5. 1.]
  [1. 5.]]]


## Test pooling layer

In [21]:
net = Net()
poolingLayer = MaxPoolingLayer((2,2), (2,4,4))
net.add_layer(poolingLayer)

input = np.array([[[1,2,1,2], [1,2,1,2], [1,2,1,2], [1,2,1,2]], [[1,2,1,2], [1,2,1,2], [1,2,1,2], [1,2,1,2]]])

print_info(input, net)

input:
[[[1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]]

 [[1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]]]
net config
Layers count: 1
layers configuration: 

MaxPooling layer
 input shape - (2, 4, 4); output shape - (2, 2, 2)
 core shape: (2, 2)



In [22]:
print(net.get_output(input))

[[[2. 2.]
  [2. 2.]]

 [[2. 2.]
  [2. 2.]]]


## Test dense layer

In [23]:
denseLayer = DenseLayer(ident, 10, 10)
denseLayer.set_weighs(np.eye(10))
denseLayer.set_biases(np.zeros((10, 1)))

net = Net()
net.add_layer(denseLayer)

input = np.arange(0,10)

print_info(input, net)

input:
[0 1 2 3 4 5 6 7 8 9]
net config
Layers count: 1
layers configuration: 

Dense layer
 neuron count: 10
 input shape - (10,); output shape - (10,)
 weight matrix:
     1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 
 biases vector: 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 



In [24]:
net.get_output(input)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

## Test reformat layer

In [25]:
net = Net()
reformatLayer = ReformatLayer((2, 4, 4))
net.add_layer(reformatLayer)

input = np.ones((2,4,4))

print_info(input, net)

input:
[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]]
net config
Layers count: 1
layers configuration: 

Reformat layer
 input shape - (2, 4, 4); output shape - (16,)



In [26]:
net.get_output(input)

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

## Test softmax layer

In [27]:
net = Net()
softmaxLayer = SoftmaxLayer(10)
net.add_layer(softmaxLayer)

input = np.arange(1, 11)
print_info(input, net)

input:
[ 1  2  3  4  5  6  7  8  9 10]
net config
Layers count: 1
layers configuration: 

Softmax layer
 input shape - (10,); output shape - (10,)



In [28]:
output = net.get_output(input)
print(output)
print(np.sum(output))

[7.80134161e-05 2.12062451e-04 5.76445508e-04 1.56694135e-03
 4.25938820e-03 1.15782175e-02 3.14728583e-02 8.55520989e-02
 2.32554716e-01 6.32149258e-01]
1.0
